In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from collections import defaultdict
from glob import glob
from skimage import io, transform
from google.colab import drive
import string 

drive.mount('/content/drive')

Mounted at /content/drive


## Read Original Data

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/ENGI 4800/data/df_converted.csv', index_col=False).dropna()[['pii','Section_content','Converted_Section_Content','Authors']]
m,n = df.shape
df.index = np.arange(m)
df.columns = ['pii','Section_content', 'Converted_Section_Content','Authors']

df.info()
df_copy = df.copy()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16949 entries, 0 to 16948
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   pii                        16949 non-null  object
 1   Section_content            16949 non-null  object
 2   Converted_Section_Content  16949 non-null  object
 3   Authors                    16949 non-null  object
dtypes: object(4)
memory usage: 662.1+ KB


In [ ]:
df[:5]

,pii,Section_content,Converted_Section_Content,Authors
0,S0001457520316912,"Qinghong Chen: Conceptualization, Data - proce...","#57221096756 : Conceptualization, Data - proc...","[{Authorseq"":1,""auid"":57221096756,""given_name_..."
1,S000145752031770X,The authors confirm contribution to the paper ...,The authors confirm contribution to the paper ...,"[{Authorseq"":1,""auid"":57221288000,""given_name_..."
2,S0001457521002001,The authors confirm contribution to the paper ...,The authors confirm contribution to the paper ...,"[{Authorseq"":1,""auid"":57208572807,""given_name_..."
3,S0001706X21001960,"Jacques D Charlwood designed the study, establ...","Jacques D #7006426873 designed the study, es...","[{Authorseq"":1,""auid"":7006426873,""given_name_p..."
4,S0002914920301132,"Conceptualization: KG, NSB; Data curation: KG,...","Conceptualization: #57198261180 , #550706125...","[{Authorseq"":1,""auid"":57198261180,""given_name_..."


## Split Content into txt 

In [ ]:
import re

df_spliited = df_copy

splitted_texts = []
sentences_count = []
for index, row in df_spliited.iterrows():

    # add paragraph sentences count 
    content = row[2]
    
    replaced_txt = re.sub(r"([^A-Z])([.|;]\s+)", r"\1\n", content)
    splitted_txt = re.split(r"\n", replaced_txt)
    
    sentences_count.append(len(splitted_txt)) 
    splitted_texts.append(splitted_txt)
    
df_spliited.insert(1,'Sentences_count', sentences_count)
df_spliited.insert(2,'Splitted_txt', splitted_texts)

In [ ]:
df_spliited[:5]

,pii,Sentences_count,Splitted_txt,Section_content,Converted_Section_Content,Authors
0,S0001457520316912,6,"[ #57221096756 : Conceptualization, Data - pro...","Qinghong Chen: Conceptualization, Data - proce...","#57221096756 : Conceptualization, Data - proc...","[{Authorseq"":1,""auid"":57221096756,""given_name_..."
1,S000145752031770X,5,[The authors confirm contribution to the paper...,The authors confirm contribution to the paper ...,The authors confirm contribution to the paper ...,"[{Authorseq"":1,""auid"":57221288000,""given_name_..."
2,S0001457521002001,5,[The authors confirm contribution to the paper...,The authors confirm contribution to the paper ...,The authors confirm contribution to the paper ...,"[{Authorseq"":1,""auid"":57208572807,""given_name_..."
3,S0001706X21001960,1,"[Jacques D #7006426873 designed the study, e...","Jacques D Charlwood designed the study, establ...","Jacques D #7006426873 designed the study, es...","[{Authorseq"":1,""auid"":7006426873,""given_name_p..."
4,S0002914920301132,9,"[Conceptualization: #57198261180 , #55070612...","Conceptualization: KG, NSB; Data curation: KG,...","Conceptualization: #57198261180 , #550706125...","[{Authorseq"":1,""auid"":57198261180,""given_name_..."


## Remove Rows with only 1 sentence  
possible cases for sentences with only one sentence, 
1. content doesn't use punctuation
2. the content uses "all authors contributes same... "

## Remove DR. MR. MS. 

In [ ]:
df_spliited.drop(df_spliited[df_spliited['Sentences_count'] == 1].index, inplace=True)
df_spliited.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15010 entries, 0 to 16948
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   pii                        15010 non-null  object
 1   Sentences_count            15010 non-null  int64 
 2   Splitted_txt               15010 non-null  object
 3   Section_content            15010 non-null  object
 4   Converted_Section_Content  15010 non-null  object
 5   Authors                    15010 non-null  object
dtypes: int64(1), object(5)
memory usage: 820.9+ KB


In [ ]:
df_spliited[:5]

,pii,Sentences_count,Splitted_txt,Section_content,Converted_Section_Content,Authors
0,S0001457520316912,6,"[ #57221096756 : Conceptualization, Data - pro...","Qinghong Chen: Conceptualization, Data - proce...","#57221096756 : Conceptualization, Data - proc...","[{Authorseq"":1,""auid"":57221096756,""given_name_..."
1,S000145752031770X,5,[The authors confirm contribution to the paper...,The authors confirm contribution to the paper ...,The authors confirm contribution to the paper ...,"[{Authorseq"":1,""auid"":57221288000,""given_name_..."
2,S0001457521002001,5,[The authors confirm contribution to the paper...,The authors confirm contribution to the paper ...,The authors confirm contribution to the paper ...,"[{Authorseq"":1,""auid"":57208572807,""given_name_..."
4,S0002914920301132,9,"[Conceptualization: #57198261180 , #55070612...","Conceptualization: KG, NSB; Data curation: KG,...","Conceptualization: #57198261180 , #550706125...","[{Authorseq"":1,""auid"":57198261180,""given_name_..."
5,S0002914920306883,7,"[ #57189385591 , MD: Conceptualization, Roles/...","Waqas Ullah, MD: Conceptualization; Roles/Writ...","#57189385591 , MD: Conceptualization; Roles/W...","[{Authorseq"":1,""auid"":57189385591,""given_name_..."


## Remove All Author Name in Content

In [ ]:
sentences_all = []

for index, row in df_spliited.iterrows():
    sentences_all += row[2]
df_sentences = pd.DataFrame(sentences_all, columns =['Original_txt'])

In [ ]:
df_sentences.info()
df_sentences.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93277 entries, 0 to 93276
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Original_txt  93277 non-null  object
dtypes: object(1)
memory usage: 728.9+ KB


,Original_txt
0,"#57221096756 : Conceptualization, Data - proc..."
1,"#57221100279 : Conceptualization, Data - proce..."
2,"#15841581600 : Conceptualization, revising-ori..."
3,"#55599484000 : Conceptualization, revising-ori..."
4,#57189246657 : Methodology


## Sentence Cleaning and Pre-processing
Pre process to get sentences list for Term Search

Pre cleanning 
1. remove author id
2. remove stopwords and punctuations
3. Stem sentence after removing author id stopwords and punctuations
term search target:


In [ ]:
# remove author id 

authorsId_removed = []
for index, row in df_sentences.iterrows():
    authorsId_removed.append(re.sub('#[\d]+', '', row[0]))
    
df_sentences.insert(1,'AuthorsId_removed', authorsId_removed)

In [ ]:
df_sentences.head(5)

,Original_txt,AuthorsId_removed
0,"#57221096756 : Conceptualization, Data - proc...",": Conceptualization, Data - processing & ana..."
1,"#57221100279 : Conceptualization, Data - proce...",": Conceptualization, Data - processing & anal..."
2,"#15841581600 : Conceptualization, revising-ori...",": Conceptualization, revising-original draft"
3,"#55599484000 : Conceptualization, revising-ori...",": Conceptualization, revising-original draft"
4,#57189246657 : Methodology,: Methodology


In [ ]:
# remove stopwords and punctuations

import nltk
import string

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)
nltk.download("stopwords", quiet=True)
stop_words = set(stopwords.words('english'))

punctuations = string.punctuation

punctuation_stopWord_removed = []

for index, row in df_sentences.iterrows():
    sentence = row[1]
    sentence = sentence.translate(str.maketrans('', '', string.punctuation.replace('#','')))
    filtered_sentence = [w for w in word_tokenize(sentence) if not w.lower() in stop_words]
    punctuation_stopWord_removed.append(' '.join(filtered_sentence))
    
df_sentences.insert(2,'punctuation_stopWord_removed', punctuation_stopWord_removed) 
    

In [ ]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
df_stem = df_sentences.copy()
stemed = []

for i, row in df_sentences.iterrows():
    sentence = df_sentences['punctuation_stopWord_removed'][i].split()
    stemed.append(' '.join([porter.stem(word) for word in sentence]))
    
df_sentences.insert(3,'stemmed', stemed) 


In [ ]:
re.fullmatch(r'[\d]+', '123') == None

False

In [ ]:
df_sentences_copy = df_sentences.copy()
df_sentences_copy.drop(df_sentences[(df_sentences['stemmed'] == '') ].index, inplace=True)
df_sentences.info()
df_sentences_copy.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 93277 entries, 0 to 93276
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Original_txt                  93277 non-null  object
 1   AuthorsId_removed             93277 non-null  object
 2   punctuation_stopWord_removed  93277 non-null  object
 3   stemmed                       93277 non-null  object
dtypes: object(4)
memory usage: 5.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 90939 entries, 0 to 93276
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Original_txt                  90939 non-null  object
 1   AuthorsId_removed             90939 non-null  object
 2   punctuation_stopWord_removed  90939 non-null  object
 3   stemmed                       90939 non-null  object
dtypes: object(4)
memory usage: 3.5+ MB


In [ ]:
df_sentences_copy.to_csv('/content/drive/MyDrive/ENGI 4800/data/All_Sentences.csv')

## Term Search ---Software

In [ ]:
#filter standard
df_matched = df_sentences_copy.loc[df_sentences_copy.stemmed.str.contains('softwa',na=False) | 
                              df_sentences_copy.stemmed.str.contains('cod',na=False) | 
                              df_sentences_copy.stemmed.str.contains('algo',na=False)]

In [ ]:
df_matched.info()
df_matched.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1577 entries, 34 to 93207
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Original_txt                  1577 non-null   object
 1   AuthorsId_removed             1577 non-null   object
 2   punctuation_stopWord_removed  1577 non-null   object
 3   stemmed                       1577 non-null   object
dtypes: object(4)
memory usage: 61.6+ KB


,Original_txt,AuthorsId_removed,punctuation_stopWord_removed,stemmed
34,"software, #6602655040","software,",software,softwar
54,"#57211321440 : Investigation, Software",": Investigation, Software",Investigation Software,investig softwar
56,"#57212222512 : Validation, Software",": Validation, Software",Validation Software,valid softwar
60,"#57226096205 : Software, Validation, Visualiza...",": Software, Validation, Visualization",Software Validation Visualization,softwar valid visual
198,"#57190834705 : Conceptualisation, Methodology...",": Conceptualisation, Methodology, Software, ...",Conceptualisation Methodology Software Formal ...,conceptualis methodolog softwar formal analysi...


In [ ]:
df_unmatched = df_sentences_copy.loc[~(df_sentences_copy.stemmed.str.contains('softwa',na=False) |
                                  df_sentences_copy.stemmed.str.contains('cod',na=False) | 
                                  df_sentences_copy.stemmed.str.contains('algo',na=False))]

In [ ]:
df_unmatched.info()
df_unmatched.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89258 entries, 0 to 93276
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Original_txt                  89258 non-null  object
 1   AuthorsId_removed             89258 non-null  object
 2   punctuation_stopWord_removed  89258 non-null  object
 3   stemmed                       89258 non-null  object
dtypes: object(4)
memory usage: 3.4+ MB


,Original_txt,AuthorsId_removed,punctuation_stopWord_removed,stemmed
0,"#57221096756 : Conceptualization, Data - proc...",": Conceptualization, Data - processing & ana...",Conceptualization Data processing analysis Met...,conceptu data process analysi methodolog write...
1,"#57221100279 : Conceptualization, Data - proce...",": Conceptualization, Data - processing & anal...",Conceptualization Data processing analysis,conceptu data process analysi
2,"#15841581600 : Conceptualization, revising-ori...",": Conceptualization, revising-original draft",Conceptualization revisingoriginal draft,conceptu revisingorigin draft
3,"#55599484000 : Conceptualization, revising-ori...",": Conceptualization, revising-original draft",Conceptualization revisingoriginal draft,conceptu revisingorigin draft
4,#57189246657 : Methodology,: Methodology,Methodology,methodolog


In [ ]:
# df_matched_sample = df_matched.sample(n=100)
# df_unmatched_sample = df_unmatched.sample(n=105)

# df_matched_sample.to_csv('/content/drive/MyDrive/ENGI 4800/data/Software_Matched_Sntences_unlabel.csv')
# df_unmatched_sample.to_csv('/content/drive/MyDrive/ENGI 4800/data/Software_Unmatched_Sentences.csv')